In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))


<b><font size= "6" color="red">ch14. 웹데이터 수집</font></b>
# 1절. BeautifulSoup 과 parser
`pip install bs4` 아나콘다를 설치하면 자동설치되는 패키지 7500개에 포험
- 공식Documentation: https://beautiful-soup-4.readthedocs.io/en/latest/


In [160]:
# 내 local PC에 파일을 url(http://~~)처럼 접근 
import requests # http 요청 처리하는 lib
from requests_file import FileAdapter # file:// 프로토콜을 다루기 위함


In [161]:
s= requests.Session() #HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
#file:// 경로로 들어오면 이 요청은 c:(local file)을 접근
response = s.get('file:///aii/source/01_Python/data/ch14_sample.html')
response

<Response [200]>

In [162]:
if response:
    print('잘 접근했습니다')
else:
    print('접근을 못했습니다.')

잘 접근했습니다


In [163]:
response.status_code
    # 200: 정상
    # 404: 없는 페이지
    # 406: get, post 오류

200

In [164]:
response.content # 바이너리 형식의 html내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [165]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [166]:
response.text # HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [167]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

In [168]:
# soup.select_one ('선택자'): 해당선택자 처음 하나만
el= soup.select_one('h1') #처음 나오는 h1태그 하나만
print('el: ', el)
print('el.text', el.text)
print('el.string', el.text)
print('el의 속성들: ', el.attrs)
print('el의 id속성: ', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id속성: ', el.attrs.get('id'))
print('el의 id속성: ', el.attrs.get('href'))
print('el의 태그이름: ', el.name)

el:  <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text Hello, CSS
el.string Hello, CSS
el의 속성들:  {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성:  text
el의 id속성:  text
el의 id속성:  None
el의 태그이름:  h1


In [169]:
# soup.select('선택자'): 해당 선택자 모두
els= soup.select('h1') # h1태그를 list
print('els: ', els)
print('els의 text들: ', [el.text for el in els])
print('els의 string들: ', [el.string for el in els])
print('els의 속성들: ' ,[el.attrs for el in els])
print('els의 class 속성들: ',[el.attrs.get('class') for el in els])

els:  [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들:  ['Hello, CSS', 'Hi, CSS']
els의 string들:  ['Hello, CSS', 'Hi, CSS']
els의 속성들:  [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들:  [['greeting', 'css'], ['css']]


In [170]:
# soup.find(태그, 속성): soup.select_one('선택자') 와 유사
print('select_one: ', soup.select_one('h1.css'))
print('find: ',soup.find('h1',{'class':'css'})) # soup.select_one('h1.css')
print('find: ',soup.find('h1',class_='css'))
print()
print('select_one: ', soup.select_one('h1#text'))
print('find',soup.find('h1',{'id','text'}))

select_one:  <h1 class="greeting css" id="text">Hello, CSS</h1>
find:  <h1 class="greeting css" id="text">Hello, CSS</h1>
find:  <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one:  <h1 class="greeting css" id="text">Hello, CSS</h1>
find None


In [171]:
# soup.find_all(태그, 속성): soup.select('선택지') 와 유사
print('select: ',soup.select('h1.css,span'))
print('find_all:',soup.find_all('h1',class_='css'))

select:  [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all: [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [172]:
print('select: ',soup.select('h1.css,span'))
print('find_all: ', soup.find_all(['h1','span'],class_='css'))
print('find_all: ',soup.find_all('h1',class_='css'))

select:  [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all:  [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all:  [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [173]:
# 없는 엘리먼트 찾기
print('select(빈 list): ', soup.select('a.css'))
print('select_one(None): ',soup.select_one('a.css'))
print('find_all(빈 list): ',soup.find_all('a',{'class','css'}))
print('find(None): ', soup.find('a',{'class':'css'}))

select(빈 list):  []
select_one(None):  None
find_all(빈 list):  []
find(None):  None


# 2절. 정적 웹 데이터 수집(정적 웹 크롤링)
## 2.1 Beautiful Soup을 활용한 html 웹 데이터 수집
### 1) 환율정보 가져오기(네이버 증권 -> 시장지표)
 - https://finance.naver.com/marketindex/

In [174]:
# 크롤링 허용범위는 사이트마다 다르다 ~/robots.txt에서 확인
    # Allow: /  - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [175]:
# 방법1 
import requests
from bs4 import BeautifulSoup
url='https://finance.naver.com/marketindex/'
response= requests.get(url)
response

soup= BeautifulSoup(response.text, 'html.parser')
soup


<script language="javascript" src="/template/head_js.naver?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20251111183746/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20251111183746/js/jindo.1.5.3.element-text-patch.js" type="text/javascript"></script>
<div id="container" style="padding-bottom:0px;">
<div class="market_include">
<div class="market_data">
<div class="market1">
<div class="title">
<h2 class="h_market1"><span>환전 고시 환율</span></h2>
</div>
<!-- data -->
<div class="data">
<ul class="data_lst" id="exchangeList">
<li class="on">
<a class="head usd" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_up">
<span class="value">1,467.50</span>
<span class="txt_krw

In [176]:
# 방법2
from urllib.request import urlopen
url='https://finance.naver.com/marketindex/'
response= urlopen(url)
# response.status
# response.read().decode('cp949')
soup= BeautifulSoup(response, 'html.parser')

In [145]:
# div.head_info > span.value (find함수)
prices= []
headinfos= soup.find_all('div', class_= 'head_info')
for headinfo in headinfos:
    price= headinfo.find('span', class_='value')
#     print(price.text.replace(',',''))
    prices.append(float(''.join(price.text.split(','))))
print(prices)

[1466.6, 950.42, 1698.03, 205.98, 153.98, 1.16, 1.318, 99.31, 61.04, 1705.7, 4116.3, 194911.34]


In [146]:
# div.head_info > span.value (select함수)
prices= soup.select('div.head_info > span.value')
print([float(p.text.replace(',',''))for p in prices])

[1466.6, 950.42, 1698.03, 205.98, 153.98, 1.16, 1.318, 99.31, 61.04, 1705.7, 4116.3, 194911.34]


In [147]:
titles= soup.select('h3.h_lst > span.blind')
for t in titles:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [201]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째에 '' 추가
units


['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [202]:
statuses= soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

하락	하락	하락	하락	상승	하락	상승	하락	상승	상승	상승	상승	

In [203]:
len(titles), len(prices), len(wons), len(statuses)

(12, 12, 1, 12)

In [204]:
for idx in range(len(titles)):
    print(f"{titles[idx].text}:{prices[idx].text}{wons[idx]}- {statuses[idx].text}")

미국 USD:1,451.20원- 하락


IndexError: string index out of range

In [205]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

미국 USD : 1,451.20원 -하락
일본 JPY(100엔) : 942.61원 -하락
유럽연합 EUR : 1,678.02원 -하락
중국 CNY : 203.81원 -하락
달러/일본 엔 : 153.4200엔 -상승
유로/달러 : 1.1564달러 -하락
영국 파운드/달러 : 1.3161달러 -상승
달러인덱스 : 99.4700 -하락
WTI : 59.75달러 -상승
휘발유 : 1699.21원 -상승
국제 금 : 4009.8달러 -상승
국내 금 : 190133.68원 -상승


### 2) 이번 주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin
```
1197회 (2025년 11월 08일 추첨)


```

In [206]:
from urllib.request import urlopen
url='https://dhlottery.co.kr/gameResult.do?method=byWin'
response= urlopen(url)
# response.status
# response.read().decode('cp949')
lotto= BeautifulSoup(response, 'html.parser')
lotto


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="동행복권" id="utitle" name="title"/>
<meta content="동행복권 1197회 당첨번호 1,5,7,26,28,43+30. 1등 총 13명, 1인당 당첨금액 2,205,089,827원." id="desc" name="description"/>
<title>로또6/45 - 회차별 당첨번호</title>
<title>동행복권</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/images/common/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/common/favicon.ico" rel="icon" type="image/x-icon"/>
<script src="/js/jquery-1.9.1.min.js" type="text/javascript"></script>
<script src="/js/jquery-ui.js" type="text/javascript"></script>
<script charset="utf-8" src="/js/common.js" type="text/javascript"></script>
<script type="text/javascript">

fn_g_init_message("");

var gameUserId = "";

function goGame(){
	var userId = "";
	
	if(userId == '' || userId == null){
		alert("로그인 후 사용 해주시기 바랍니다.");
		location.href = "/user.do?method=login";
		return;
	}
	
	$.ajax({
		type:"get",                          

In [207]:
times = lotto.select_one('div.win_result strong').text
date  = lotto.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = lotto.select('div.num.win  span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(lotto.select_one('div.num.bonus > p > span').text)
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
no title               link
0  한풀 꺾인 비트코인  https://v.daum.net/v/20251110094711892

In [208]:
# 방법1
word='비트코인'
url= 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8'+word
print(url)
response= requests.get(url)
response
soup= BeautifulSoup(response.text, "html.parser")


https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8비트코인


In [209]:
# 방법2
from urllib.parse import quote
word=quote('비트코인') # url에 한글이 들어가면 안되니, quote로 한글을 바꿔준다
url= 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8'+word

response= urlopen(url)
print(url)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


In [210]:


items_find_list=[] #검색한 결과를 담을 리스트 (딕셔너리 list)

items_el= soup.select('div.item-title> strong.tit-g > a')
for idx, item in enumerate(items_el):
    print(idx, item.text, item.attrs.get('href'))
    items_find_list.append([idx, item.text, item.attrs['href']])
import pandas as pd
pd.DataFrame(items_find_list,columns=['번호','제목','링크'])

0  "한풀 꺾인 비트코인, 상승세 곧 회복"…근거는?  http://v.daum.net/v/20251110094711892
1  “지금 들어가?”… 트럼프 관세 배당 발언에 10만달러 회복한 비트코인  http://v.daum.net/v/20251110114112103
2  트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]  http://v.daum.net/v/20251110112101913
3  "비트코인 21억 간다더니"...갑자기 말 바꿨다  http://v.daum.net/v/20251109105545139
4  비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]  http://v.daum.net/v/20251110103033108
5  美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]  http://v.daum.net/v/20251110111435519
6  비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?  http://v.daum.net/v/20251110092814031
7  “비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다  http://v.daum.net/v/20251110094507791
8  비트코인 소폭 상승, 10만 달러대 '방어선' 유지  http://v.daum.net/v/20251110082746613
9  [가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까  http://v.daum.net/v/20251109112746909


,번호,제목,링크
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,“지금 들어가?”… 트럼프 관세 배당 발언에 10만달러 회복한 비트코인,http://v.daum.net/v/20251110114112103
2,2,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
3,3,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
6,6,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
7,7,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
8,8,"비트코인 소폭 상승, 10만 달러대 '방어선' 유지",http://v.daum.net/v/20251110082746613
9,9,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909


In [211]:
items_find_list=[] #검색한 결과를 담을 리스트
#div.item-title > strong.tit-g > a
item_titles= soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a= item.find('a')
#     print(idx, a.text)
#     items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({'no':idx,'title':a.text, 'link':a.attrs['href'],})
pd.DataFrame(items_find_list)

,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,“지금 들어가?”… 트럼프 관세 배당 발언에 10만달러 회복한 비트코인,http://v.daum.net/v/20251110114112103
2,2,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
3,3,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
6,6,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
7,7,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
8,8,"비트코인 소폭 상승, 10만 달러대 '방어선' 유지",http://v.daum.net/v/20251110082746613
9,9,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909


In [219]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)

def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스 검색한 결과를 출력 -> list로 return '
#     url=(f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}')
    response= requests.get(url)
    print(response.status_code)

In [220]:
collect_news_list('청바지',2)

200


In [237]:
def collect_news_list(keyword, page):
    url= f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q': keyword, 'p': page}
    response= requests.get(url, params=params)
    soup= BeautifulSoup(response.text, 'html.parser')
    items_find_list=[]
    item_titles= soup.select('div.item-title > strong.tit-g > a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [238]:
collect_news_list('AI',1)

[[0,
  ' 국내 최대 AI 컨펙스 ‘AI 서밋 서울 & 엑스포’ 개막 ',
  'http://v.daum.net/v/20251110112049892'],
 [1, ' AI 거품론 속…오픈AI, 백악관에 SOS ', 'http://v.daum.net/v/20251110042102550'],
 [2,
  ' 또 퍼지는 ‘AI거품론’… 커지는 ‘AI워싱’ 경보 ',
  'http://v.daum.net/v/20251109183027380'],
 [3,
  ' [뉴욕증시] AI 거품론 속 급락…AI 선도주 엔비디아·팔란티어 폭락 ',
  'http://v.daum.net/v/20251105064248616'],
 [4,
  ' 5.5년만에 AI 박사학위 취득...내년 AI인재양성에 1.4조 예산 투입 ',
  'http://v.daum.net/v/20251110142138252'],
 [5,
  ' "도메인 지식 없는 AI는 공허"···최태원, 본업 기반 AI 경쟁력 강조 ',
  'http://v.daum.net/v/20251110144852927'],
 [6,
  " 와이즈넛, 경기도 'AI 업무비서' 개발…공공 AI사업 본격화 ",
  'http://v.daum.net/v/20251110135541028'],
 [7,
  ' AI 동료 시대 전문가 시대 끝나…"불완전성이 AI와 다른 인간의 가치" ',
  'http://v.daum.net/v/20251110102052624'],
 [8,
  " 중견회계법인협의회, AI 감사 솔루션 '오딘 AI' 도입 ",
  'http://v.daum.net/v/20251110150515967'],
 [9,
  ' 국내 최대 AI 컨펙스 행사 개막…무협, 코엑스·DMK글로벌과 AI 전시회 공동 개최 ',
  'http://v.daum.net/v/20251110154346873']]

In [245]:
import time
keywords=['청바지','동대문']
result0= [] #청바지 1~3페이지까지 검색한 결과를 담을 list
result1= [] #동대문 1~3페이지까지 검색한 결과를 담을 list
pages= 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어{keyword}검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)
        

===1번째 검색어청바지검색 결과 수집 중입니다===
===2번째 검색어동대문검색 결과 수집 중입니다===


In [250]:
result0_df= pd.DataFrame(result0, columns=['no','title','link'])
result1_df= pd.DataFrame(result1, columns=['no','title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 핫플된 사연? [르포]",http://v.daum.net/v/20251110100357735
1,1,"동대문구, ‘11월 자전거교실’ 개강… 올해 마지막 초보자 교육 진행",http://v.daum.net/v/20251110145114101
2,2,"음주운전 차, 동대문역서 인도 돌진...日 관광객 '사망'",http://v.daum.net/v/20251103073316480
3,3,"동대문구, 25일까지 '주민 건강도시학교'···""건강하게 살자""",http://v.daum.net/v/20251109223318356
4,4,"서울 동대문구, 15일 배봉산정원 가족축제 개최",http://v.daum.net/v/20251104104517122


In [253]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False)
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User 
- 자동으로 브라우저를 통해 요청하는 것 처럼 보이기

In [266]:
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글 인코딩 처리
url= f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
response= urlopen(url)
response.status
soup= BeautifulSoup(response, 'html.parser')
headers={'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request= Request(url, headers=headers)

response= urlopen(request)
soup= BeautifulSoup(response, 'html.parser')
soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


<!DOCTYPE html>

<html lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="text/html;charset=utf-8" http-equiv="content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="off" name="autocomplete"/>
<meta content="always" name="referrer"/>
<meta content="telephone=no,address=no,email=no" name="format-detection">
<meta content="비트코인 – Daum 검색" property="og:title"/>
<meta content="https://search.daum.net/search?w=news&amp;nil_search=btn&amp;DA=PGD&amp;enc=utf8&amp;q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8" property="og:url"/>
<meta content="Daum 검색에서 비트코인에 대한 최신정보를 찾아보세요." property="og:description"/>
<meta content="https://search1.daumcdn.net/search/statics/common/img/20250107/daum_og.png" property="og:image"/>
<meta content="다음검색" property="og:site_name"/>
<link href="https://t1.daumcdn.net/top/favicon/20241223/daum.ico" rel="shortcut icon"/>
<link href="//sl.search.daum.net/" rel="preconnect"/>
<lin

- https://www.melon.com/chart/index.htm
    https://www.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [267]:
# 방법1
import requests
url='https://www.melon.com/chart/index.htm'
melonpage= requests.get(url)
melonpage

<Response [406]>

In [270]:
# 방법2
from urllib.request import urlopen, Request
url='https://www.melon.com/chart/index.htm'
headers= {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request= Request(url, headers=headers)
melonpage= urlopen(request)
soup= BeautifulSoup(melonpage, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>멜론차트&gt;TOP100&gt;멜론</title>
<meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
<meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
<meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
<meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
<meta content="4022717807957185" property="fb:app_id"/>
<meta content="Melon" property="og:title"/>
<meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
<meta content="음악이 필요한 순간, 멜론" property="og:description"/>
<meta content="http://www.melon.com/chart/index.htm" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="멜론" prope

In [288]:
# 방법1: User-Agent 추가
url='https://www.melon.com/chart/index.htm'
headers= {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage= requests.get(url, headers=headers)
soup= BeautifulSoup(melonpage.text, #melonpage.content
                   'html.parser')
ranks= soup.select('td div.wrap.t_center> span.rank')
titles= soup.select('div.ellipsis.rank01 > span> a') #title.text.strip()
singers= soup.select('div.ellipsis.rank02') # singer.text.strip()[:20] 
for rank, title, singer in zip(ranks, titles, singers):
    print(f"{rank.text}위| {title.text.strip()} - {singer.text.strip()[:20]}")

1위| Blue Valentine - NMIXXNMIXX
2위| 타임캡슐 - 다비치다비치
3위| Golden - HUNTR/X, EJAE, AUDRE
4위| Drowning - WOODZWOODZ
5위| Good Goodbye - 화사 (HWASA)화사 (HWASA)
6위| SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7위| 달리 표현할 수 없어요 - 로이킴로이킴
8위| 뛰어(JUMP) - BLACKPINKBLACKPINK
9위| Soda Pop - KPop Demon Hunters C
10위| 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11위| FAMOUS - ALLDAY PROJECTALLDAY
12위| 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
13위| IRIS OUT - Kenshi YonezuKenshi 
14위| XOXZ - IVE (아이브)IVE (아이브)
15위| 모르시나요(PROD.로코베리) - 조째즈조째즈
16위| Rich Man - aespaaespa
17위| Hollywood Action - BOYNEXTDOORBOYNEXTDO
18위| 한번 더 이별 - 이창섭이창섭
19위| 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
20위| 너에게 닿기를 - 10CM10CM
21위| HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위| 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위| body - 다영 (DAYOUNG)다영 (DAYO
24위| toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25위| Whiplash - aespaaespa
26위| 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27위| 천상연 - 이창섭이창섭
28위| 나는 반딧불 - 황가람황가람
29위| 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AKMU (악뮤)AKMU (악뮤)
30위| 청춘만화 - 

### 5) 네이버 지식인으로 검색(open API사용X)
- 특정 keyword를 특정페이지 수만큼

In [6]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword='쳇지피티티'
# url= f'https://kin.naver.com/search/list.naver'
# params= {'query':keyword}
# response=get(url, params=params)
url=f'https://kin.naver.com/search/list.naver?query={keyword}'
print(url)
response= get(url)
print(response.status_code)
soup= BeautifulSoup(response.text, 'html.parser')

https://kin.naver.com/search/list.naver?query=쳇지피티티
200


In [9]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword= quote('쳇지피티')
url= f'https://kin.naver.com/search/list.naver?query={keyword}'
response= urlopen(url)
print(response.status)
soup= BeautifulSoup(response,'html.parser')


200


In [20]:
# 페이징 포함
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword= quote('쳇지피티')
pages= 3
items_list = [] #크롤링한 데이터를 담을 list (title, link)
for page in range(1, pages+1):
    url= f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response= urlopen(url)
#     print(response.status)
    soup=BeautifulSoup(response, 'html.parser')
    items= soup.select('dt> a')
    for item in items:
        title= item.text
        link= item.attrs.get('href')
        items_list.append({'title':title, 'link':link})
        
df= pd.DataFrame(items_list)
df

쳇지피티 사용 확인 AI


## 2.2 openAPI사용: json웹데이터 수집
### 1) 네이버 지식인으로 검색(open API사용 O)
- 네이버 개발자센터에서 애플리케이션 등록(이름/ 검색/ WEB:http://localhost)
- .env 파일에 CLIENT_ID/ CLIENT_SECRET 환경변수 저장
- 환경변수를 읽기 위해서 `pip install dotenv` 
- 특정 keyword를 지식검색(데이터수 30개)

In [21]:
! pip install dotenv

In [24]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
    #dotenv_path='.env'
            ) # 현소스와 같은 폴더내의 .env를 메모리에 load
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [56]:
# 방법2
# 네이버 검색 API 예제 - 블로그 검색
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("쳇지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과

# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers={
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
request= urllib.request.Request(url, headers=headers)

response = urllib.request.urlopen(request)

rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(type(json.loads(response_body.decode('utf-8'))))
    #response_body의 문자 내용을  json스타일의 딕셔너리로 변환
    print(json.loads(response_body.decode('utf-8')))
else:
    print("Error Code:" + rescode)

<class 'dict'>
{'lastBuildDate': 'Tue, 11 Nov 2025 12:31:38 +0900', 'total': 3143, 'start': 1, 'display': 10, 'items': [{'title': '<b>쳇지피티</b> 사주 정확성', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31501&docId=489876742&qb=7LOH7KeA7ZS87Yuw&enc=utf8', 'description': '<b>쳇지피티</b> 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요 안녕하세요. 요즘 지피티에 대해 물어보시는 분들이 많아서 저도 해봤지만 아직 생년월일도ㅠ파악하지 못합니다. 유료버전 무료버전 둘 다... '}, {'title': '<b>쳇지피티</b> 관해서', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10704&docId=488200030&qb=7LOH7KeA7ZS87Yuw&enc=utf8', 'description': '<b>쳇지피티</b>한테 질문 제대로 하려면 어떻게 해야 하냐요?... <b>쳇지피티</b> 한테 하는질문도 1.우리 이야기 만들래)?(호칭.... <b>쳇지피티</b> 한테 어떻게 질문해야 정확한 답변을 들을수 있죠 <b>쳇지피티</b> 앱이 편하시다 하신분들은 편하신이유... '}, {'title': '사주에서 뭐가 부족한지 봐주세요(<b>쳇지피티</b> 답변 금지)', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31501&docId=488905210&qb=7LOH7KeA7ZS87Yuw&enc=utf8', 'description': '사주에서 뭐가 부족한지 봐주세요(<b>쳇지피티</b> 답변 금지) 남매입니다 남자 1986년 9월 8일 양력 오후 5시 여자 1992년 10월 31일

In [74]:
#방법 1
import requests
import pandas as pd
import json # response 텍스트를 json스타일의 딕셔너리로 
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query ="쳇지피티"
headers={
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f"https://openapi.naver.com/v1/search/kin?query={query}&display=30"
# response= requests.get(url, headers=headers)
url= 'https://openapi.naver.com/v1/search/kin'
params= {'query':query, 'display':30}
response= requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# json.loads(response.text)['items']
items= response.json()['items']
items_list=[]
for item in items:
    title= item['title'].replace('<b>','').replace('</b>','')
    link= item['link']
    description= item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link,description])
# pd.DataFrame(items_list, columns=['title','link','description'])
item

{'title': '<b>쳇지피티</b> 오류',
 'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10303&docId=484943564&qb=7LOH7KeA7ZS87Yuw&enc=utf8',
 'description': '유료결제하고 사용중인데 오류가 나며서 계속 사용할수가 없네요 일주일 정도 된거 같고 14일까지 사용할수있어요 사진 감사합니다! 이런 오류가 반복되면 정말 답답하실 수밖에 없죠 &quot;흠.. 뭔자 잘못된것 같습니다&quot;라는 메시지와... '}

### quiz) 네이버 open API 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv출력
```
title link

       제목, 링크, 썸네일, sizeheight, sizewidth
```

In [101]:
#방법 1
import requests
import pandas as pd
import json # response 텍스트를 json스타일의 딕셔너리로 
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

query ="청바지"
headers={
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}

url= 'https://openapi.naver.com/v1/search/image'
params= {'query':query, 'display':100}
response= requests.get(url, params=params, headers=headers)
response
# print(response.text)
items= response.json()['items']
items_list=[]
for item in items:
    title= item.get('title')
    link= item.get('link')
    thumbnail= item.get('thumbnail')
    
    items_list.append([title, link,thumbnail, sizeheight,sizewidth])
df= pd.DataFrame(items_list,columns=['제목','링크','썸네일','sizeheight','sizewidth'])

In [93]:
df.to_csv(f'data/ch14_청바지',index=False)

In [103]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    file_extension=link.split('.')[-1]
    quote_index=file_extension.find('?') #확장자 뒤에 ?가 있는 위치
    if quote_index != -1:
        file_extension=file_extension[:quote_index]
    img= requests.get(link).content
    with open(f'data/image/{attr}_{idx+1}_{query}.{file_extension}','wb') as f:
        f.write(img)
save_image('메인',0,df.loc[0,'링크'],'청바지')

In [106]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = [] # 정보가 담길 리스트
    for idx, item in enumerate(items):
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('data/image/ch14_청바지.csv')
df.sample()

= = = 20% 진행완료 = = =
= = = 40% 진행완료 = = =
= = = 60% 진행완료 = = =
= = = 80% 진행완료 = = =


,제목,링크,썸네일,sizeheight,sizewidth
99,남성 가을 겨울 기모 도톰한 스트레이트핏 스판 청바지 E15Z49E 젠2 : 수:피아,http://shop1.phinf.naver.net/20251012_221/1760...,https://search.pstatic.net/common/?type=b150&s...,800,800


## 2.3 XML웹데이터 수집
- RSS / open API을 통한 XML 웹데이터 수집
### 1) 전국 날씨 RSS를 Beautipul Soup을 이용한 크롤링
- 구글에 "기상청 RSS" 검색한 첫번째 사이트 이동

In [116]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list=[]
url='https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
#방법1
# target= requests.get(url)
# soup= BeautifulSoup(target.text, 'xml') #pip install lxml 필요없음

# 방법2
target= urlopen(url)
soup= BeautifulSoup(target, 'xml')
local_tas= soup.select('local_ta')
for local_ta in local_tas:
    local_name= local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange= local_ta.select_one('week1_local_ta_similarRange').text
    week1_local_ta_minVal=local_ta.select_one('week1_local_ta_minVal').text
    week1_local_ta_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_local_ta_maxVal= local_ta.select_one('week1_local_ta_maxVal').text
    
    items_list.append({
        '지역':local_name,
        '1주 평년기온':week1_normalYear,
        '1주 기온범위':week1_similarRange,
        '1주 낮을확률':week1_local_ta_minVal,
        '1주 비슷할 확률':week1_local_ta_similarVal,
        '1주 높은확률':week1_local_ta_maxVal
    })
df= pd.DataFrame(items_list)
df['1주 평년기온'] = df['1주 평년기온'].astype(np.float64)
df['1주 낮을확률'] = df['1주 낮을확률'].astype(np.int16)
df['1주 비슷할 확률'] = df['1주 비슷할 확률'].astype(np.int16)
df['1주 높은확률'] = df['1주 높은확률'].astype(np.int16)
df.head()

,지역,1주 평년기온,1주 기온범위,1주 낮을확률,1주 비슷할 확률,1주 높은확률
0,"전국(제주도,북한제외)",5.9,5.0~6.8,20,40,40
1,서울ㆍ인천ㆍ경기도,4.8,3.7~5.9,20,40,40
2,강원도 영서,2.6,1.5~3.7,20,40,40
3,강원도 영동,5.8,4.9~6.7,20,40,40
4,대전ㆍ세종ㆍ충청남도,5.4,4.5~6.3,20,40,40


### 2) xml응답하는 open API활용
- data.go.kr에서
    - 서울특별시_노선정보 조회 서비스(버스ID, 정류장목록과정류장ID)
    - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

In [117]:
# STEP1 버스 번호의 버스 id 받아오기
# 서울특별시_노선정보 조회 서비스

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('KEY'))
# print(os.getenv('KEY1'))

True

In [70]:
import requests
from bs4 import BeautifulSoup
# urlretrieve(url, 저장경로) : url파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote
# busNum= quote('종로07')
busNum= '341'
key=os.getenv('KEY')
url= f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
savefilename1= 'data/ch14_busInfo.xml'

urlretrieve(url, savefilename1) #xml 파일(url) savefilename으로 저장
with open(savefilename1, encoding='UTF-8') as f:
    xml=f.read();
soup=BeautifulSoup(xml, 'xml')

In [72]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
import os

busNum = '341'
key = os.getenv('KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url1)
soup = BeautifulSoup(response.text, 'xml')

busRouteId = None  # 기본값 지정 (없을 때 NameError 방지)

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break;
print('busRouteId= ',busRouteId)

busRouteId=  100100056


In [73]:
#step2. 버스 id로 버스정류장목록받아오기(정류장명, 정류장idㅡ 경도, 위도)
# 서울

In [74]:
import pandas as pd
key = os.getenv('KEY')
url2=f"http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}"
response=requests.get(url2)
soup= BeautifulSoup(response.content,'xml')
itemLists= soup.find_all('itemList')
print(f"{busNum}번정류장 갯수:{len(itemLists)}")
bus_station=[]
for itemList in itemLists:
    stationNm= itemList.find('stationNm').text
    station= itemList.find('station').text
    gpsX= itemList.find('gpsX').text
    gpsY= itemList.find('gpsY').text
    bus_station.append([stationNm,station,gpsX,gpsY])
df_station=pd.DataFrame(bus_station, columns=['정류소명','id','경도','위도'])
df_station.head()

341번정류장 갯수:93


,정류소명,id,경도,위도
0,강동공영차고지,124000001,127.173885,37.564066
1,고덕리엔파크1단지,124000003,127.1704714177,37.5611595214
2,해뜨는주유소,124000005,127.170805,37.559102
3,고덕천교차로,124000007,127.171147352,37.5563241348
4,고덕자이.고덕천물놀이장,124000009,127.1720516501,37.5534666908


In [75]:
df_station.to_csv(f'data/ch14_노선정보.csv',index=False)

In [76]:
pd.options.display.max_rows= 60

In [77]:
# step3 버스id로 실시간 위치정보를 받아오기
# 서울특별시

In [78]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
# print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemLists)}대입니다')
bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text # 차량번호
    congetion = itemList.select_one('congetion').text # 혼잡도(congetion)
    # 0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion = '없음' if congetion=='0' \
            else '여유' if congetion=='3' \
            else '보통' if congetion=='4' \
            else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX= itemList.select_one('gpsX').text
    gpsY= itemList.select_one('gpsY').text
    
    lastStnId= itemList.select_one('lastStnId').text
    nextStId= itemList.select_one('nextStId').text
    nextStTm= itemList.select_one('nextStTm').text
    
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion,
        '경도':gpsX,
        '위도':gpsY,
        '최종정류소id':lastStnId,
        '다음정류소id':nextStId,
        '도착소요시간':nextStTm
    })
df_position = pd.DataFrame(bus_position)
df_position

운행중인 버스는 19대입니다


,차량번호,혼잡도,경도,위도,최종정류소id,다음정류소id,도착소요시간
0,서울70사2769,여유,127.172409,37.563055,124000001,124000357,1418
1,서울74사1579,여유,127.148972,37.536822,124000360,124000357,476
2,서울75사2849,여유,127.125026,37.538225,124000453,123000043,1244
3,서울74사5089,여유,127.10995,37.516666,123000039,123000043,249
4,서울74사5012,여유,127.082925,37.511787,123000053,122000098,746
5,서울74사5039,여유,127.06513,37.509563,122000093,122000098,147
6,서울70사2773,여유,127.058076,37.507417,122000098,122000114,211
7,서울70사2723,여유,127.041764,37.5024,122000174,122000184,3667
8,서울74사1572,여유,127.038337,37.478055,121000215,122000184,2397
9,서울74사6235,여유,127.021219,37.463922,121000294,122000184,1540


In [79]:
df_station.loc[df_station['id']=='124000362','정류소명'].iloc[0]

'상일초교'

In [131]:
def station_name(row):
    row= row.copy()
    'row의 최종 정류소명과 다음 정류소 명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] =df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] =df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    minite= int(row['도착소요시간']) // 60
    sec= int(row['도착소요시간'])%60

    row['도착소요시간'] = f'{minite}분{sec}초'
    return row
    
# station_name(df_position.loc[20])

In [133]:
df= df_position.apply(station_name, axis=1)
df.drop(['최종정류소id','다음정류소id'], axis=1)

,차량번호,혼잡도,경도,위도,도착소요시간,이전정류소명,다음정류소명
0,서울70사2769,여유,127.172409,37.563055,23분38초,강동공영차고지,강동역
1,서울74사1579,여유,127.148972,37.536822,7분56초,강동자이.프라자아파트,강동역
2,서울75사2849,여유,127.125026,37.538225,20분44초,천호역,잠실역8번출구
3,서울74사5089,여유,127.10995,37.516666,4분9초,신천동진주아파트,잠실역8번출구
4,서울74사5012,여유,127.082925,37.511787,12분26초,잠실엘스아파트앞,한국무역센터.삼성역
5,서울74사5039,여유,127.06513,37.509563,2분27초,강남경찰서.강남운전면허시험장,한국무역센터.삼성역
6,서울70사2773,여유,127.058076,37.507417,3분31초,한국무역센터.삼성역,포스코건너편
7,서울70사2723,여유,127.041764,37.5024,61분7초,서울상록회관.한국기술센터,강남역1번출구.역삼세무서
8,서울74사1572,여유,127.038337,37.478055,39분57초,말죽거리공원사거리,강남역1번출구.역삼세무서
9,서울74사6235,여유,127.021219,37.463922,25분40초,형촌마을입구(서초네이처힐1.2단지),강남역1번출구.역삼세무서


## Quiz1.
- yes24 의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를수집해서 파일에 저장하세요

    * 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller
    * ch14_yes24.csv
        - 순위, 책이름, 저장및 출판사, 가격

In [181]:
# 방법1 
import requests
from bs4 import BeautifulSoup
url='https://www.yes24.com/product/category/bestseller?categoryNumber=001'
response= requests.get(url)
response

soup= BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE html >

<html lang="ko">
<head><link href="https://m.yes24.com/Home/Best?DispNo=001" media="only screen and(max-width: 640px)" rel="alternate"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="dpr, width, viewport-width, rtt, downlink, ect, UA, UA-Platform, UA-Arch, UA-Model, UA-Mobile, UA-Full-Version" http-equiv="Accept-CH"/>
<meta content="86400" http-equiv="Accept-CH-Lifetime"/>
<meta content="unsafe-url" name="referrer"/>
<meta content="width=1170" name="viewport"/>
<title> 국내도서 종합 베스트 - 예스24 </title>
<meta content="예스24" name="title"/>
<meta content="YOUR EVERY STORY 문화 콘텐츠 플랫폼, 예스24" name="description"/>
<meta content="인터넷 서점, 온라인 쇼핑, 상품 추천, 쇼핑몰, 상품 검색, 도서 정보, 국내도서, 외국도서, 전자책, eBook, 이북, 크레마, 공연, 콘서트, 뮤지컬, 음반, 예매, DVD, 블루레이, 예스24, YES24, 교보문고, 알라딘, 리센스, 예스24 도서용품, 친환경 PB 브랜드, 사은품, 굿즈" name="keywords"/>
<link href="//www.yes24.com/OpensearchDescription.xml" rel="search" title="YES

In [316]:
print(url)

https://www.yes24.com/product/category/bestseller?categoryNumber=001


In [336]:

ranks= soup.select('div.itemUnit em.ico.rank')
titles= soup.select('div.info_row.info_name a.gd_name')
authors = soup.select('div.info_row.info_pubGrp span.authPub.info_auth a')

prices= soup.select('div.info_row.info_price em.yes_b')
pubs= soup.select('div.info_row.info_pubGrp span.authPub.info_pub a')

itemList=[]

for t, r, a, p, p1 in zip(titles, ranks, authors, prices, pubs):
    
    itemList.append({ '순위':r.text, '제목':t.text,'저자':a.text.replace('\r','').strip(), '출판사':p1.text,'가격':p.text, })

# 확인
print(itemList[:5])


[{'순위': '1', '제목': '트렌드 코리아 2026', '저자': '김난도', '출판사': '미래의창', '가격': '18,000'}, {'순위': '2', '제목': '처음 만나는 양자의 세계', '저자': '전미영', '출판사': '북플레저', '가격': '17,100'}, {'순위': '3', '제목': '대형주 추세추종 투자법칙', '저자': '최지혜', '출판사': '사피엔테스', '가격': '19,800'}, {'순위': '4', '제목': '위버멘쉬', '저자': '권정윤', '출판사': '떠오름', '가격': '16,020'}, {'순위': '5', '제목': '박곰희 연금 부자 수업', '저자': '한다혜', '출판사': '인플루엔셜', '가격': '18,900'}]


In [337]:
df= pd.DataFrame(itemList)

In [338]:
df.head(20)

,순위,제목,저자,출판사,가격
0,1,트렌드 코리아 2026,김난도,미래의창,"18,000"
1,2,처음 만나는 양자의 세계,전미영,북플레저,"17,100"
2,3,대형주 추세추종 투자법칙,최지혜,사피엔테스,"19,800"
3,4,위버멘쉬,권정윤,떠오름,"16,020"
4,5,박곰희 연금 부자 수업,한다혜,인플루엔셜,"18,900"
5,6,주식 투자의 뿌리,정보 더 보기/감추기,애덤스미스,"19,800"
6,7,절창 切創,김난도,문학동네,"16,200"
7,8,다크 심리학,전미영,어센딩,"19,710"
8,9,단 한 줄만 내 마음에 새긴다고 해도,최지혜,포레스트북스,"21,420"
9,10,어른의 행복은 조용하다,권정윤,페이지2북스,"16,020"


In [307]:
df.to_csv(f'data/ch14_yes24.csv',index=False)

In [341]:
# import
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [342]:
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.replace('\r','').strip() for authors_el in authors_els]
    for i, author in enumerate(authors):
        match = re.search(r"(.*)외 \d+명", author)
        if(match):
            authors[i] = match.group()
            print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

0 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름
10 번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름


,순위,책이름,저자,출판사,가격
0,1,트렌드 코리아 2026,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명",미래의창,"18,000원"
